In [110]:
import pandas as pd
import numpy as np
from geopy import distance as distance
from geopy.geocoders import Nominatim
import datetime as dt
from datetime import datetime
path = '/Users/peter/Downloads/202210-citibike-tripdata.csv'

In [265]:
df = pd.read_csv(path, dtype={"end_station_id":"str","start_station_id":"str"})


In [266]:
df_1 = df

In [267]:
#Know what you're deleting
print(len(df['start_station_id'][df['start_station_id'].str.contains("SYS")==True]))
print(len(df['start_station_id'][df['start_station_id'].str.contains("Lab")==True]))
print(len(df['start_station_id'][df['start_station_id'].str.contains("JC")==True]))
print(len(df['start_station_id'][df['start_station_id'].str.contains("HB")==True]))
print(len(df['end_station_id'][df['end_station_id'].str.contains("SYS")==True]))
print(len(df['end_station_id'][df['end_station_id'].str.contains("Lab")==True]))
print(len(df['end_station_id'][df['end_station_id'].str.contains("JC")==True]))
print(len(df['end_station_id'][df['end_station_id'].str.contains("HB")==True]))

28
1
0
0
157
1
74
65


In [275]:
dtypess={"ride_id": "str", "rideable_type": "str","started_at":"str"
,"ended_at":"str","start_station_name":"str","start_station_id":"str"
,"end_station_name":"str","end_station_id":"str","start_lat":"float"
,"start_lng":"float","end_lat":"float","end_lng":"float","member_casual":"str"}

#Removing Aforementioned Rows
df = df[df['start_station_id'].str.contains("SYS")==False]
df = df[df['start_station_id'].str.contains("Lab")==False]
df = df[df['start_station_id'].str.contains("JC")==False]
df = df[df['start_station_id'].str.contains("HB")==False]
df = df[df['end_station_id'].str.contains("HB")==False]
df = df[df['end_station_id'].str.contains("SYS")==False]
df = df[df['end_station_id'].str.contains("Lab")==False]
df = df[df['end_station_id'].str.contains("JC")==False]

#string coerce:
#df['start_station_id']=df['start_station_id'].apply(lambda x: x.str.extract('(\d+)'))
#df['start_station_id'] = pd.to_numeric(df['start_station_id'],errors='coerce')
#df['end_station_id'] = pd.to_numeric(df['end_station_id'],errors='coerce')

#reassigning datatypes, printing the column in the loop in the event of errors
for key, value in dtypess.items():
    print(key)
    df[key] =  df[key].astype(value)

ride_id
rideable_type
started_at
ended_at
start_station_name
start_station_id
end_station_name
end_station_id
start_lat
start_lng
end_lat
end_lng
member_casual


In [290]:
#Creating Dictionary 
u_df_st = df.drop_duplicates(subset=['start_station_id']).dropna()
u_df_st = u_df_st[['start_station_id','start_lat','start_lng']].dropna()
u_df_en = df.drop_duplicates(subset=['end_station_id']).dropna()
u_df_en = u_df_en[['start_station_id','start_lat','start_lng']].dropna()
#if ending location is not in the starting set, we add it to the first df
#unique_stations_end = u_df_en['start_station_id'].unique().tolist()
#unique_stations_end = df['end_station_id'].unique().tolist()
#unique_stations_end_cl = [x for x in unique_stations_end if str(x) != 'nan']
#unique_stations_end_1 = [x for x in unique_stations_end_cl if x not in unique_stations]

In [314]:
target = pd.DataFrame(0,index = u_df_st['start_station_id'], columns = u_df_st['start_station_id'])
print(target.shape)
target.head(20)

(1632, 1632)


start_station_id,7756.10,4458.06,5164.05,7382.04,5159.04,4010.13,6331.01,4939.07,6115.09,6860.12,...,6167.03,8061.01,5074.08,7034.09,6191.04,4253.02,7719.13,8632.07,3019.02,6514.08
start_station_id,,,,,,,,,,,,,,,,,,,,,
7756.10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4458.06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5164.05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7382.04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5159.04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4010.13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6331.01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4939.07,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6115.09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [315]:
import time
start = time.time()

for i in range(len(u_df_st['start_station_id'])):
    for j in range(len(u_df_st['start_station_id'])):
        if i == j:
            target.iloc[i,j] = 0.0
        elif target.iloc[j,i] > 0:
            target.iloc[i,j] = target.iloc[j,i]
        else:
            coord_1 = (u_df_st.iloc[i,1],u_df_st.iloc[i,2])
            coord_2 = (u_df_st.iloc[j,1],u_df_st.iloc[j,2])
            target.iloc[i,j] = distance.distance(coord_1,coord_2).miles

end = time.time()
print(end-start)

900.2439849376678


In [319]:
target.to_csv('/Users/peter/Downloads/pairwise_station_distances.csv')

In [308]:
print(target.shape, u_df_st.shape)

(1634, 1634) (1632, 3)


In [307]:
u_df_st[u_df_st['start_station_id'] == '8095.06']

,start_station_id,start_lat,start_lng
45007,8095.06,40.832171,-73.889686


In [189]:
df = df_original.iloc[0:5,:]

In [193]:
df = df_original
df.shape

(719658, 13)

In [190]:
#df_original = df
#df = df_original
df = df[df['end_lat'].notna()]
#data cleaning pipeline
#time_diffs
#df['started_at_new'] = datetime.strftime(df['started_at'],'%Y-%M-%D %H:%M:%S')
df['started_at'] = df['started_at'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df['ended_at'] = df['ended_at'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df['length_trip_minutes'] = df.apply(lambda x: (x['ended_at'] - x['started_at']).total_seconds()/60.0, axis = 1)
#member, non-member encoding
df['member_casual_en'] = df['member_casual'].apply(lambda x: 1 if x == 'member' else 0)
df['s_unique_lat_lon'] = list(zip(df['start_lat'],df['start_lng']))
df['e_unique_lat_lon'] = list(zip(df['end_lat'],df['end_lng']))

def distance_between_stat(x):
    dist = []
    for i in range(len(x['start_lat'])):
        coord_1 = (x['start_lat'][i],x['start_lng'][i])
        coord_2 = (x['end_lat'][i],x['end_lng'][i])
        dist.append(distance.distance(coord_1,coord_2).miles)
    return dist

def geopy_address_query_start(x):
    geolocator = Nominatim(user_agent="geoapiExercises")
    zips = []
    addresses = []
    for i in range(len(x['start_lat'])):
        coord_1 = (x['start_lat'][i],x['start_lng'][i])
        str_res = geolocator.reverse(coord_1)
        str_zip = str_res.raw['address']['postcode']
        str_add = str_res.raw['address']['suburb']
        zips.append(str_zip)
        #print(str_res.raw['address'])
        addresses.append(str_add)
    return addresses,zips

#df['length_of_trip_miles'] = distance_between_stat(df)

df['city'], df['zip_code'] = geopy_address_query_start(df)

#df_for_parsing = df[['start_station_id','end_station_id','s_unique_lat_lon','e_unique_lat_lon']]



In [107]:
#subsetting unique stations

#unique_guider_frames#
u_df_st = df.drop_duplicates(subset=['start_station_id'])

unique_stations = df['start_station_id'].unique().tolist()
unique_stations_end = df['end_station_id'].unique().tolist()
unique_stations_end = [x for x in unique_stations_end if x not in unique_stations]
unique_stations.extend(unique_stations_end)
#board to dict
lat_long_stations = {}


In [191]:
df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,length_trip_minutes,member_casual_en,s_unique_lat_lon,e_unique_lat_lon,city,zip_code
0,2CF35A89B21C6E3C,classic_bike,2022-10-24 14:30:25,2022-10-24 14:37:35,St. Nicholas Ave & W 126 St,7756.10,Amsterdam Ave & W 136 St,7920.05,40.811388,-73.951995,40.819746,-73.951819,member,7.166667,1,"(40.811388254, -73.95199453800001)","(40.819746, -73.951819)",Manhattan,10027
1,291CC2C6D361C2DA,classic_bike,2022-10-30 03:53:38,2022-10-30 03:57:57,Stuyvesant Ave & Gates Ave,4458.06,Halsey St & Tompkins Ave,4319.07,40.688445,-73.933195,40.682369,-73.944118,member,4.316667,1,"(40.688444972, -73.93319451800001)","(40.682368700000005, -73.944118)",Brooklyn,11221
2,80192DC8429B07EE,classic_bike,2022-10-08 15:03:06,2022-10-08 15:17:13,Broadway & Berry St,5164.05,Meserole Ave & Manhattan Ave,5666.04,40.710361,-73.965304,40.727086,-73.952991,member,14.116667,1,"(40.71036129922581, -73.96530389785768)","(40.72708584, -73.95299117)",Brooklyn,11249
3,D214E550C317C65E,classic_bike,2022-10-18 08:32:04,2022-10-18 08:36:41,W 84 St & Columbus Ave,7382.04,W 76 St & Columbus Ave,7281.09,40.785000,-73.972834,40.780184,-73.977285,member,4.616667,1,"(40.78499979, -73.97283406)","(40.7801839724239, -73.97728532552719)",Manhattan,10024
4,2D23E8BAE75AA184,classic_bike,2022-10-26 08:00:40,2022-10-26 08:10:27,Pike St & Monroe St,5159.04,Broad St & Bridge St,4962.08,40.711731,-73.991930,40.703652,-74.011678,member,9.783333,1,"(40.71173107, -73.99193043)","(40.70365182, -74.01167797)",Manhattan,10002
